In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## Q1. Downloading the data

In [8]:
# Read the data for January. How many columns are there?

data= pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
len(data.columns)


19

## Q2. Computing duration

In [12]:
# Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 
data['duration'] = data.tpep_dropoff_datetime - data.tpep_pickup_datetime
data


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,0 days 00:13:59
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,0 days 00:19:27
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,0 days 00:24:31
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,0 days 00:13:00


In [13]:
data.duration = data.duration.dt.total_seconds() / 60


In [14]:
# What's the standard deviation of the trips duration in January?
data['duration'].describe()
#std      4.259435e+01


count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

## Q3. Dropping outliers

In [15]:
old_data = len(data)


In [16]:
data = data[(data.duration >= 1) & (data.duration <= 60)]
new_data = len(data)


In [17]:
result=new_data / old_data * 100
result


98.1220282212598

## Q4. One-hot encoding

In [21]:
from sklearn.feature_extraction import DictVectorizer


In [18]:
categorical = ['PULocationID', 'DOLocationID']
data[categorical] = data[categorical].astype(str)


C:\Users\sudwa\AppData\Local\Temp\ipykernel_3740\461318387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[categorical] = data[categorical].astype(str)


In [19]:
train_dicts = data[categorical].to_dict(orient='records')
train_dicts[:5]


[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [22]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape


(3009173, 515)

## Q5. Training a model

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


In [24]:
y_train = data.duration.values

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [25]:
y_pred = model.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)


7.6492619382463465

## Q6. Evaluating the model


In [37]:
def read_data(filename):
    df=pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df

df_train = read_data('./data/yellow_tripdata_2023-01.parquet')
df_val= read_data('./data/yellow_tripdata_2023-02.parquet')


In [38]:
len(df_train), len(df_val)


(3009173, 2855951)

In [39]:
categorical = ['PULocationID', 'DOLocationID']
train_dicts = df_train[categorical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')


In [40]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)


In [41]:
y_train = df_train['duration'].values
y_val = df_val['duration'].values


In [43]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)

mean_squared_error(y_train, y_train_pred, squared=False)


9.835277713631873

In [44]:
y_val_pred = lr.predict(X_val)
mean_squared_error(y_val, y_val_pred, squared=False)


9.963607595829973

In [45]:
las = Lasso(0.01)
las.fit(X_train, y_train)

y_pred = las.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)


9.963607818498394

In [46]:
rig = Ridge()
rig.fit(X_train, y_train)

y_pred = rig.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)


9.963607595830075

In [47]:
import joblib
joblib.dump(lr,'model.joblib')


['model.joblib']